In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, show, draw
import scipy.io
import imageio
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
from IPython.display import clear_output

%matplotlib inline

# Filter by Threshold

### Discard boxes that have lower probability than the set threshold

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
    
    box_scores = box_confidence * box_class_probs 
    box_classes = K.argmax(box_scores, axis = -1) # returns 19*19,5,1 if you have the index of the max, you have the class for that box
    box_classes_score = K.max(box_scores, axis = -1) # returns 19*19,5,1, max value for that box
    
    filtering_mask = (box_classes_score >= threshold) # this reduces the boxes per cell
    
    scores = tf.boolean_mask(box_classes_score, filtering_mask)# returns 19*19,?,1 scores depending on how many out of 5 boxes per cell is above the threshold
    boxes = tf.boolean_mask(boxes, filtering_mask) # returns 19*19,?,4 boxes the coordinates of the boxes that have made it after filtering
    classes = tf.boolean_mask(box_classes, filtering_mask) # returns 19*19,?,1 classes corresponding to the highest predicted to the surviving boxes
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

# Non-max suppression NMS
### Reduce the remaining/similar boxes by NMS

### IoU
Interseciton over Union(IoU) is used for NMS 

ex. after threshold filtering from the previous step
1. pick a box that have the highest probability of an object
2. compute IoU of other boxes to the previously chosen box

if IoU is higher than a set threshold, then most likely these boxes identify the same object

To show the idea, IoU is defined below but is already part of `tf.image.non_max_suppression`

In [ ]:
def iou(box1, box2):
    
    #origin is located at upper left corner
    
    xi1 = max(box1[0],box2[0])
    yi1 = max(box1[1],box2[1])
    xi2 = min(box1[2],box2[2])
    yi2 = min(box1[3],box2[3])
    
    inter_area = max((xi2-xi1),0) * max((yi2-yi1),0) # compute intersection area, 0 if negative area. By convention used, 2nd coordinate should be greater than the 1st
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area/union_area # overlap computation, 1 if overlaps completely with each other, 0 if not
    
    return iou

In [ ]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou = " + str(iou(box1, box2)))

### NMS

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold)
    
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

## Evaluate output from pretrained YOLO with the functions defined above

In [ ]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes = 10, score_threshold = 0.6, iou_threshold = 0.5):
    
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy, box_wh) # convert pretrained output (midpoint, width, height) to (upperleft corner, lowerright corner)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    boxes = scale_boxes(boxes, image_shape)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

# Pretrained YOLO

In [ ]:
sess = K.get_session()

In [ ]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.) #what image? is this the image to predict boxes?, or the image where YOLO was pretrained

In [ ]:
!gsutil cp gs://steel-index-252802/yolo.h5 model_data/

In [ ]:
yolo_model = load_model("model_data/yolo.h5")

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
def predict(sess, image_file):
    image, image_data = preprocess_image("images/"+image_file, model_image_size = (608,608))
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict = {yolo_model.input: image_data, K.learning_phase():0})
    print("Found {} boxes for {}".format(len(out_boxes), image_file))
    colors = generate_colors(class_names)
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors )
    image.save(os.path.join("out", image_file), quality = 90)
    output_image = imageio.imread(os.path.join("out", image_file))
    #imshow(output_image)
    
    return output_image, out_scores, out_boxes, out_classes

In [ ]:
files = [i for i in sorted(os.listdir("images/"))]
for file in files:
    clear_output(wait=True)
    out_image, out_scores, out_boxes, out_classes = predict(sess, file)
    imshow(out_image)
    show()